In [1]:
print('hi')

hi


Project Phase 1: Stepwise API Exploration

Step 1: Import Libraries


In [2]:
!pip install requests pandas
import requests
import pandas as pd
import json


Step 2: Simple API Query for Studies on a Condition

In [3]:
base_url = "https://clinicaltrials.gov/api/v2/studies"
params = {
    "query.cond": "Diabetes",   # You can change 'Diabetes' to any condition
    "pageSize": 20              # Small batch for testing – you can increase later
}
response = requests.get(base_url, params=params)
print("Status code:", response.status_code)


Status code: 200


Step 3: Check Top-Level Keys and Count

In [4]:
if response.status_code == 200:
    data = response.json()
    studies = data.get('studies', [])
    print(f"Number of studies returned: {len(studies)}")
    print("Top-level keys in first study:", list(studies[0].keys()))
else:
    print("API error:", response.text)


Number of studies returned: 20
Top-level keys in first study: ['protocolSection', 'derivedSection', 'hasResults']


Step 4: See All Available Fields & Their Values in One Study

In [5]:
# Print the complete contents of the first study, indented for readability:
print(json.dumps(studies[0], indent=2))


{
  "protocolSection": {
    "identificationModule": {
      "nctId": "NCT00767208",
      "orgStudyIdInfo": {
        "id": "CRNHRA-06-002"
      },
      "organization": {
        "fullName": "Centre de Recherche en Nutrition Humaine Rhone-Alpe",
        "class": "OTHER"
      },
      "briefTitle": "Diabetes and Metabolic Postprandial Responses",
      "officialTitle": "Comparison of Postprandial Metabolic Responses Between Type 2 Diabetic and Healthy Overweight Subjects"
    },
    "statusModule": {
      "statusVerifiedDate": "2009-05",
      "overallStatus": "COMPLETED",
      "expandedAccessInfo": {
        "hasExpandedAccess": false
      },
      "startDateStruct": {
        "date": "2005-01"
      },
      "primaryCompletionDateStruct": {
        "date": "2006-05",
        "type": "ACTUAL"
      },
      "completionDateStruct": {
        "date": "2008-05",
        "type": "ACTUAL"
      },
      "studyFirstSubmitDate": "2008-10-06",
      "studyFirstSubmitQcDate": "2008-10-06

Explore All Fields in One Study

In [6]:
print(json.dumps(studies[0], indent=2))


{
  "protocolSection": {
    "identificationModule": {
      "nctId": "NCT00767208",
      "orgStudyIdInfo": {
        "id": "CRNHRA-06-002"
      },
      "organization": {
        "fullName": "Centre de Recherche en Nutrition Humaine Rhone-Alpe",
        "class": "OTHER"
      },
      "briefTitle": "Diabetes and Metabolic Postprandial Responses",
      "officialTitle": "Comparison of Postprandial Metabolic Responses Between Type 2 Diabetic and Healthy Overweight Subjects"
    },
    "statusModule": {
      "statusVerifiedDate": "2009-05",
      "overallStatus": "COMPLETED",
      "expandedAccessInfo": {
        "hasExpandedAccess": false
      },
      "startDateStruct": {
        "date": "2005-01"
      },
      "primaryCompletionDateStruct": {
        "date": "2006-05",
        "type": "ACTUAL"
      },
      "completionDateStruct": {
        "date": "2008-05",
        "type": "ACTUAL"
      },
      "studyFirstSubmitDate": "2008-10-06",
      "studyFirstSubmitQcDate": "2008-10-06

 Compare Field Availability Across Multiple Studies

In [7]:
for i in range(3):  # Try first 3 studies
    print(f"\n--- Study {i+1} ---")
    print(json.dumps(studies[i], indent=2)[:1200]) # Print first ~1200 chars for readability



--- Study 1 ---
{
  "protocolSection": {
    "identificationModule": {
      "nctId": "NCT00767208",
      "orgStudyIdInfo": {
        "id": "CRNHRA-06-002"
      },
      "organization": {
        "fullName": "Centre de Recherche en Nutrition Humaine Rhone-Alpe",
        "class": "OTHER"
      },
      "briefTitle": "Diabetes and Metabolic Postprandial Responses",
      "officialTitle": "Comparison of Postprandial Metabolic Responses Between Type 2 Diabetic and Healthy Overweight Subjects"
    },
    "statusModule": {
      "statusVerifiedDate": "2009-05",
      "overallStatus": "COMPLETED",
      "expandedAccessInfo": {
        "hasExpandedAccess": false
      },
      "startDateStruct": {
        "date": "2005-01"
      },
      "primaryCompletionDateStruct": {
        "date": "2006-05",
        "type": "ACTUAL"
      },
      "completionDateStruct": {
        "date": "2008-05",
        "type": "ACTUAL"
      },
      "studyFirstSubmitDate": "2008-10-06",
      "studyFirstSubmitQcD

Summarize Key Fields & Value Types

In [8]:
records = []
for study in studies:
    protocol = study.get('protocolSection', {})
    id_module = protocol.get('identificationModule', {})
    status_module = protocol.get('statusModule', {})
    conds_module = protocol.get('conditionsModule', {})
    brief_title = id_module.get('briefTitle', '')
    nct_id = id_module.get('nctId', '')
    status = status_module.get('overallStatus', '')
    conditions = ', '.join(conds_module.get('conditions', []))
    records.append({
        'nct_id': nct_id,
        'brief_title': brief_title,
        'status': status,
        'conditions': conditions
    })
study_df = pd.DataFrame(records)
print(study_df.head())


        nct_id                                        brief_title     status  \
0  NCT00767208      Diabetes and Metabolic Postprandial Responses  COMPLETED   
1  NCT01572740  Efficacy and Safety of Liraglutide in Combinat...  COMPLETED   
2  NCT01147718  A Drug Interaction Study of Albiglutide and Di...  COMPLETED   
3  NCT01182480  Chronic Care Management/Patient Relationship M...  COMPLETED   
4  NCT03880162  Metabolic Effects of a Low Carbohydrate Versus...  COMPLETED   

                                          conditions  
0                        Type 2 Diabetes, Overweight  
1                Diabetes, Diabetes Mellitus, Type 2  
2                          Diabetes Mellitus, Type 2  
3                                  Diabetes Mellitus  
4  PreDiabetes, Obesity, Morbid, Bariatric Surger...  


Data Quality & Distribution Analysis

In [9]:
print(study_df.status.value_counts())
print(study_df.conditions.value_counts())
print(study_df.isna().sum())


status
COMPLETED     15
UNKNOWN        2
TERMINATED     1
WITHDRAWN      1
RECRUITING     1
Name: count, dtype: int64
conditions
Diabetes Mellitus                                                                                                 3
Diabetes Mellitus, Type 1                                                                                         2
Type 2 Diabetes, Overweight                                                                                       1
Diabetes Mellitus, Type 2                                                                                         1
Diabetes, Diabetes Mellitus, Type 2                                                                               1
Diabetes Mellitus, Diabetic Nephropathy, Arterial Hypertension                                                    1
Diabete Type 2                                                                                                    1
Diabetic Foot Ulcer                                        

Successfully connected to ClinicalTrials.gov API

Inspected and listed study fields and example values

Built a pandas table with samples (NCT ID, Title, Status, Conditions)

Checked the spread of statuses and conditions

What's in My Table?

In [10]:
print(study_df.shape)         # (rows, columns)
print(study_df.columns.tolist())  # List all column names


(20, 4)
['nct_id', 'brief_title', 'status', 'conditions']


In [11]:
print(study_df.head(10))


        nct_id                                        brief_title      status  \
0  NCT00767208      Diabetes and Metabolic Postprandial Responses   COMPLETED   
1  NCT01572740  Efficacy and Safety of Liraglutide in Combinat...   COMPLETED   
2  NCT01147718  A Drug Interaction Study of Albiglutide and Di...   COMPLETED   
3  NCT01182480  Chronic Care Management/Patient Relationship M...   COMPLETED   
4  NCT03880162  Metabolic Effects of a Low Carbohydrate Versus...   COMPLETED   
5  NCT03274362                    Headspace Mindfulness App Trial     UNKNOWN   
6  NCT01130727  The Effect of Green Tea or Cocoa Extracts in E...   COMPLETED   
7  NCT05455242       Habit Formation for Diabetes Self-Management   COMPLETED   
8  NCT03509870  Bone Marrow Derived Allogeneic Mesenchymal Str...  TERMINATED   
9  NCT03641170  The Acute Effect of Physical Activity on Postp...   COMPLETED   

                                          conditions  
0                        Type 2 Diabetes, Overweight 

Check for Duplicate NCT IDs

In [12]:
print('Duplicate NCT IDs:', study_df.duplicated('nct_id').sum())

Duplicate NCT IDs: 0


Check for Missing Values in All Columns

In [13]:
print('Missing values per column:')
print(study_df.isnull().sum())


Missing values per column:
nct_id         0
brief_title    0
status         0
conditions     0
dtype: int64


Check Unique Value Distributions

In [14]:
print('Unique trial statuses:', study_df['status'].unique())
print('Counts per status:')
print(study_df['status'].value_counts())

print('\nUnique conditions found:')
print(study_df['conditions'].unique())
print('Counts per condition:')
print(study_df['conditions'].value_counts())


Unique trial statuses: ['COMPLETED' 'UNKNOWN' 'TERMINATED' 'WITHDRAWN' 'RECRUITING']
Counts per status:
status
COMPLETED     15
UNKNOWN        2
TERMINATED     1
WITHDRAWN      1
RECRUITING     1
Name: count, dtype: int64

Unique conditions found:
['Type 2 Diabetes, Overweight' 'Diabetes, Diabetes Mellitus, Type 2'
 'Diabetes Mellitus, Type 2' 'Diabetes Mellitus'
 'PreDiabetes, Obesity, Morbid, Bariatric Surgery Candidate'
 'Diabetes Mellitus, Diabetic Nephropathy, Arterial Hypertension'
 'Diabete Type 2' 'Diabetic Foot Ulcer' 'Gestational Diabetes'
 'Diabetes Mellitus, Type 1'
 'Non Alcoholic Fatty Liver Disease, Type 2 Diabetes, Liver Cirrhosis'
 'Diabetic Neuropathies, Diabetes Mellitus, Type 2, Polyneuropathies'
 'Healthy Subjects' 'Diabetes Mellitus, Type 1, Diabetes Mellitus, Type 2'
 'Hypoglycemia' 'Diabetes Mellitus Type 2 in Obese, Anxiety, Cardiopathy'
 'Healthy, Cancer, Heart Diseases, Neurological Diseases or Conditions, Kidney Diseases, Diabetes, Other Disease']
Counts per

Step: Add and Inspect "briefSummary"
1. Expand your table:

In [15]:
# Create a new list to capture additional info
records = []
for study in studies:
    protocol = study.get('protocolSection', {})
    id_module = protocol.get('identificationModule', {})
    status_module = protocol.get('statusModule', {})
    conds_module = protocol.get('conditionsModule', {})
    desc_module = protocol.get('descriptionModule', {})
    nct_id = id_module.get('nctId', '')
    brief_title = id_module.get('briefTitle', '')
    status = status_module.get('overallStatus', '')
    conditions = ', '.join(conds_module.get('conditions', []))
    brief_summary = desc_module.get('briefSummary', '')
    records.append({
        'nct_id': nct_id,
        'brief_title': brief_title,
        'status': status,
        'conditions': conditions,
        'brief_summary': brief_summary
    })
studies_df = pd.DataFrame(records)
print(studies_df[['nct_id','brief_title','brief_summary']].head(5))


        nct_id                                        brief_title  \
0  NCT00767208      Diabetes and Metabolic Postprandial Responses   
1  NCT01572740  Efficacy and Safety of Liraglutide in Combinat...   
2  NCT01147718  A Drug Interaction Study of Albiglutide and Di...   
3  NCT01182480  Chronic Care Management/Patient Relationship M...   
4  NCT03880162  Metabolic Effects of a Low Carbohydrate Versus...   

                                       brief_summary  
0  The aim of this trial was to compare postprand...  
1  This trial is conducted in Asia. The purpose o...  
2  This Phase I, open-label, sequential, single-c...  
3  This proof of concept study proposes to evalua...  
4  To investigate the metabolic effects of a low ...  


If You Want to Do Both Step 1 and 2 (You Mentioned)
1. Check summary length: Run this code to see if any brief summaries are much shorter or longer than others (helps spot incomplete or overly technical records):

In [16]:
print(studies_df['brief_summary'].str.len())
print('Shortest summary:', studies_df['brief_summary'].str.len().min())
print('Longest summary:', studies_df['brief_summary'].str.len().max())


0      322
1      285
2      163
3      729
4      169
5      465
6      204
7      202
8      145
9      813
10     124
11     333
12     515
13     247
14     126
15     185
16     210
17     258
18    2419
19     158
Name: brief_summary, dtype: int64
Shortest summary: 124
Longest summary: 2419


2. Inspect eligibility criteria field: Here’s code to add ‘eligibility criteria’ to your table—reveals who the study is for, critical for retrieval/personalization:

In [17]:
records = []
for study in studies:
    protocol = study.get('protocolSection', {})
    id_module = protocol.get('identificationModule', {})
    status_module = protocol.get('statusModule', {})
    conds_module = protocol.get('conditionsModule', {})
    desc_module = protocol.get('descriptionModule', {})
    eligibility_module = protocol.get('eligibilityModule', {})
    nct_id = id_module.get('nctId', '')
    brief_title = id_module.get('briefTitle', '')
    status = status_module.get('overallStatus', '')
    conditions = ', '.join(conds_module.get('conditions', []))
    brief_summary = desc_module.get('briefSummary', '')
    eligibility = eligibility_module.get('eligibilityCriteria', '')
    records.append({
        'nct_id': nct_id,
        'brief_title': brief_title,
        'status': status,
        'conditions': conditions,
        'brief_summary': brief_summary,
        'eligibility_criteria': eligibility
    })
studies_df = pd.DataFrame(records)
print(studies_df[['nct_id','brief_title','eligibility_criteria']].head(5))


        nct_id                                        brief_title  \
0  NCT00767208      Diabetes and Metabolic Postprandial Responses   
1  NCT01572740  Efficacy and Safety of Liraglutide in Combinat...   
2  NCT01147718  A Drug Interaction Study of Albiglutide and Di...   
3  NCT01182480  Chronic Care Management/Patient Relationship M...   
4  NCT03880162  Metabolic Effects of a Low Carbohydrate Versus...   

                                eligibility_criteria  
0  Inclusion Criteria:\n\n* Overweight or obese v...  
1  Inclusion Criteria:\n\n* Type 2 diabetes melli...  
2  Inclusion Criteria:\n\n* healthy volunteers\n*...  
3  Inclusion Criteria:\n\n* patients with diabete...  
4  Inclusion Criteria:\n\n* Female and male subje...  


Recommended Next Step
1. Inspect the Eligibility Criteria’s Coverage and Variety:

In [18]:
print('Eligibility text lengths:')
print(studies_df['eligibility_criteria'].str.len())
print('Shortest eligibility:', studies_df['eligibility_criteria'].str.len().min())
print('Longest eligibility:', studies_df['eligibility_criteria'].str.len().max())
print('\nFew sample eligibility texts:')
print(studies_df['eligibility_criteria'].head(3))


Eligibility text lengths:
0      908
1     1341
2      849
3      404
4      652
5      589
6      332
7      788
8     4574
9      239
10    4766
11    2063
12    1676
13     864
14     331
15     853
16    2990
17    2194
18     385
19     303
Name: eligibility_criteria, dtype: int64
Shortest eligibility: 239
Longest eligibility: 4766

Few sample eligibility texts:
0    Inclusion Criteria:\n\n* Overweight or obese v...
1    Inclusion Criteria:\n\n* Type 2 diabetes melli...
2    Inclusion Criteria:\n\n* healthy volunteers\n*...
Name: eligibility_criteria, dtype: object


Step-by-Step EDA Expansion – ClinicalTrials.gov Sample

Step 1 – Add More Fields to Your Table

In [19]:
records = []
for study in studies:
    protocol = study.get('protocolSection', {})
    id_module = protocol.get('identificationModule', {})
    status_module = protocol.get('statusModule', {})
    conds_module = protocol.get('conditionsModule', {})
    desc_module = protocol.get('descriptionModule', {})
    eligibility_module = protocol.get('eligibilityModule', {})
    sponsor_module = protocol.get('sponsorCollaboratorsModule', {})
    arms_module = protocol.get('armsInterventionsModule', {})
    outcomes_module = protocol.get('outcomesModule', {})

    nct_id = id_module.get('nctId', '')
    brief_title = id_module.get('briefTitle', '')
    status = status_module.get('overallStatus', '')
    conditions = ', '.join(conds_module.get('conditions', []))
    brief_summary = desc_module.get('briefSummary', '')
    eligibility = eligibility_module.get('eligibilityCriteria', '')
    sponsor = sponsor_module.get('leadSponsor', {}).get('name', '')
    interventions = ', '.join([i.get('name', '') for i in arms_module.get('interventions', [])])
    primary_outcomes = ', '.join([o.get('measure', '') for o in outcomes_module.get('primaryOutcomes', [])])

    records.append({
        'nct_id': nct_id,
        'brief_title': brief_title,
        'status': status,
        'conditions': conditions,
        'brief_summary': brief_summary,
        'eligibility_criteria': eligibility,
        'sponsor': sponsor,
        'interventions': interventions,
        'primary_outcomes': primary_outcomes
    })
studies_df = pd.DataFrame(records)
print(studies_df.head(5))  # See your expanded sample!


        nct_id                                        brief_title     status  \
0  NCT00767208      Diabetes and Metabolic Postprandial Responses  COMPLETED   
1  NCT01572740  Efficacy and Safety of Liraglutide in Combinat...  COMPLETED   
2  NCT01147718  A Drug Interaction Study of Albiglutide and Di...  COMPLETED   
3  NCT01182480  Chronic Care Management/Patient Relationship M...  COMPLETED   
4  NCT03880162  Metabolic Effects of a Low Carbohydrate Versus...  COMPLETED   

                                          conditions  \
0                        Type 2 Diabetes, Overweight   
1                Diabetes, Diabetes Mellitus, Type 2   
2                          Diabetes Mellitus, Type 2   
3                                  Diabetes Mellitus   
4  PreDiabetes, Obesity, Morbid, Bariatric Surger...   

                                       brief_summary  \
0  The aim of this trial was to compare postprand...   
1  This trial is conducted in Asia. The purpose o...   
2  This Phase 

Check Coverage and Variety for New Columns

In [20]:
print('\nMissing values per column:')
print(studies_df.isnull().sum())
print('\nUnique sponsors:')
print(studies_df['sponsor'].unique())
print('\nSample interventions:')
print(studies_df['interventions'].head(5))
print('\nSample primary outcomes:')
print(studies_df['primary_outcomes'].head(5))



Missing values per column:
nct_id                  0
brief_title             0
status                  0
conditions              0
brief_summary           0
eligibility_criteria    0
sponsor                 0
interventions           0
primary_outcomes        0
dtype: int64

Unique sponsors:
['Centre de Recherche en Nutrition Humaine Rhone-Alpe' 'Novo Nordisk A/S'
 'GlaxoSmithKline' 'Denver Health and Hospital Authority'
 'Insel Gruppe AG, University Hospital Bern' 'BCDiabetes.Ca'
 'University of Campinas, Brazil' 'Des Moines University'
 'Steno Diabetes Center Copenhagen' 'University of Aarhus'
 'Tolerion, Inc.' 'Hoffmann-La Roche'
 'University Hospital, Gentofte, Copenhagen'
 'Johnson & Johnson Pharmaceutical Research & Development, L.L.C.'
 'Lund University' 'Eli Lilly and Company' 'Takeda'
 'Xeris Pharmaceuticals' 'Instituto de Cardiologia do Rio Grande do Sul'
 'Chinese University of Hong Kong']

Sample interventions:
0                                    oral glucose load
1       

Save Progress and Summarize Findings

In [21]:
studies_df.to_csv('expanded_studies_eda.csv', index=False)
print('Saved expanded table to expanded_studies_eda.csv')


Saved expanded table to expanded_studies_eda.csv


You’ve expanded your EDA by adding interventions, sponsor, and primary outcomes fields to your clinical trial dataset.

You checked for missingness — all new fields are present in your sample, which is a sign of high data quality.

You looked at value variety (multiple sponsors, diverse interventions/outcomes), ensuring you’ll have rich evidence for retrieval.

You’ve saved your table for reproducibility (CSV in a /data folder).








Why These Steps Matter
This is exactly how top-tier projects start:

You are building a clear, modular, evidence-grounded database for retrieval-augmented healthcare AI.

You document field coverage and edge-case handling (sponsors, interventions, outcomes, etc.), matching the database structure and best practices described in reputable medical informatics literature and guides.​

You ensure traceability and transparency, so every chatbot answer or analytic result can connect back to a source trial (with NCT ID, sponsor, intervention, and outcome).

2. Querying Data for Two More Diseases

Step 1: Pull Asthma Trials

In [22]:
# Query for Asthma studies
base_url = "https://clinicaltrials.gov/api/v2/studies"
params = {
    "query.cond": "Asthma",
    "pageSize": 20
}
response = requests.get(base_url, params=params)
if response.status_code == 200:
    data = response.json()
    studies = data.get('studies', [])
    print(f"Returned {len(studies)} asthma studies. Top keys: {list(studies[0].keys())}")
else:
    print("API error:", response.text)


Returned 20 asthma studies. Top keys: ['protocolSection', 'derivedSection', 'hasResults']


Step 2: Pull Alzheimer's Trials

In [23]:
# Query for Alzheimer's studies
params = {
    "query.cond": "Alzheimer",
    "pageSize": 20
}
response = requests.get(base_url, params=params)
if response.status_code == 200:
    data = response.json()
    studies = data.get('studies', [])
    print(f"Returned {len(studies)} Alzheimer's studies. Top keys: {list(studies[0].keys())}")
else:
    print("API error:", response.text)


Returned 20 Alzheimer's studies. Top keys: ['protocolSection', 'derivedSection', 'hasResults']


3. EDA for New Disease Tables

In [24]:
records = []
for study in studies:
    protocol = study.get('protocolSection', {})
    id_module = protocol.get('identificationModule', {})
    status_module = protocol.get('statusModule', {})
    conds_module = protocol.get('conditionsModule', {})
    desc_module = protocol.get('descriptionModule', {})
    eligibility_module = protocol.get('eligibilityModule', {})
    sponsor_module = protocol.get('sponsorCollaboratorsModule', {})
    arms_module = protocol.get('armsInterventionsModule', {})
    outcomes_module = protocol.get('outcomesModule', {})

    nct_id = id_module.get('nctId', '')
    brief_title = id_module.get('briefTitle', '')
    status = status_module.get('overallStatus', '')
    conditions = ', '.join(conds_module.get('conditions', []))
    brief_summary = desc_module.get('briefSummary', '')
    eligibility = eligibility_module.get('eligibilityCriteria', '')
    sponsor = sponsor_module.get('leadSponsor', {}).get('name', '')
    interventions = ', '.join([i.get('name', '') for i in arms_module.get('interventions', [])])
    primary_outcomes = ', '.join([o.get('measure', '') for o in outcomes_module.get('primaryOutcomes', [])])

    records.append({
        'nct_id': nct_id,
        'brief_title': brief_title,
        'status': status,
        'conditions': conditions,
        'brief_summary': brief_summary,
        'eligibility_criteria': eligibility,
        'sponsor': sponsor,
        'interventions': interventions,
        'primary_outcomes': primary_outcomes
    })
studies_df = pd.DataFrame(records)
print(studies_df.head(5))
studies_df.to_csv('eda_phase1_newdisease.csv', index=False)
print('Saved EDA table for new disease to eda_phase1_newdisease.csv')


        nct_id                                        brief_title      status  \
0  NCT04451408  A Study of LY3372993 in Participants With Alzh...   COMPLETED   
1  NCT03472183  Exploration of the Enteric Nervous System in A...   WITHDRAWN   
2  NCT00582127  Evaluation of the COGNISION(TM) System as an E...   COMPLETED   
3  NCT07154394  Cardiac Amyloid Deposits and Heart Dysfunction...  RECRUITING   
4  NCT00104273  Rasagiline 1 mg and 2 mg Added to Aricept 10 m...   COMPLETED   

                                          conditions  \
0                         Alzheimer Disease, Healthy   
1                                  Alzheimer Disease   
2                                Alzheimer's Disease   
3  Dementia, Alzheimer Type, Mild Cognitive Impai...   
4                      Dementia, Alzheimer's Disease   

                                       brief_summary  \
0  The main purpose of this study is to evaluate ...   
1  The close homology between the central and ent...   
2  This 

Next Steps: Expand EDA to More Diseases

Step 1: Query Asthma Trials

In [25]:
params = {
    "query.cond": "Asthma",
    "pageSize": 20
}
response = requests.get(base_url, params=params)
if response.status_code == 200:
    data = response.json()
    studies = data.get('studies', [])
    print(f"Returned {len(studies)} asthma studies. Top keys: {list(studies[0].keys())}")
else:
    print("API error:", response.text)


Returned 20 asthma studies. Top keys: ['protocolSection', 'derivedSection', 'hasResults']


Step 2: Extract Asthma Study Table

In [26]:
records = []
for study in studies:
    protocol = study.get('protocolSection', {})
    id_module = protocol.get('identificationModule', {})
    status_module = protocol.get('statusModule', {})
    conds_module = protocol.get('conditionsModule', {})
    desc_module = protocol.get('descriptionModule', {})
    eligibility_module = protocol.get('eligibilityModule', {})
    sponsor_module = protocol.get('sponsorCollaboratorsModule', {})
    arms_module = protocol.get('armsInterventionsModule', {})
    outcomes_module = protocol.get('outcomesModule', {})

    nct_id = id_module.get('nctId', '')
    brief_title = id_module.get('briefTitle', '')
    status = status_module.get('overallStatus', '')
    conditions = ', '.join(conds_module.get('conditions', []))
    brief_summary = desc_module.get('briefSummary', '')
    eligibility = eligibility_module.get('eligibilityCriteria', '')
    sponsor = sponsor_module.get('leadSponsor', {}).get('name', '')
    interventions = ', '.join([i.get('name', '') for i in arms_module.get('interventions', [])])
    primary_outcomes = ', '.join([o.get('measure', '') for o in outcomes_module.get('primaryOutcomes', [])])

    records.append({
        'nct_id': nct_id,
        'brief_title': brief_title,
        'status': status,
        'conditions': conditions,
        'brief_summary': brief_summary,
        'eligibility_criteria': eligibility,
        'sponsor': sponsor,
        'interventions': interventions,
        'primary_outcomes': primary_outcomes
    })
studies_df = pd.DataFrame(records)
print(studies_df.head(5))
studies_df.to_csv('eda_phase1_asthma.csv', index=False)
print('Saved Asthma table to eda_phase1_asthma.csv')


        nct_id                                        brief_title     status  \
0  NCT02084043  In Vitro Assessment of a Breath-synchronized V...  COMPLETED   
1  NCT05774340  A Study of CM326 in Subjects With Moderate to ...    UNKNOWN   
2  NCT00279786               B2-Adrenergic Receptor Polymorphisms  COMPLETED   
3  NCT04168554  Telemedicine in the Generals Practitioners Office    UNKNOWN   
4  NCT04098094  Outcomes of RV Dysfunction in Acute Exacerbati...    UNKNOWN   

                                          conditions  \
0  Respiratory Diseases, Lung Diseases, Cystic Fi...   
1                          Moderate to Severe Asthma   
2                                 Status Asthmaticus   
3  Respiratory Distress Syndrome, Pneumonia, Bron...   
4  Acute Exacerbation of COPD, Acute Exacerbation...   

                                       brief_summary  \
0  Using an adult lung bench model of non invasiv...   
1  This study is a multi-center, randomized, doub...   
2  Beta(2)-adr

Step 3: Query Cancer Trials

In [27]:
params = {
    "query.cond": "Cancer",
    "pageSize": 20
}
response = requests.get(base_url, params=params)
if response.status_code == 200:
    data = response.json()
    studies = data.get('studies', [])
    print(f"Returned {len(studies)} cancer studies. Top keys: {list(studies[0].keys())}")
else:
    print("API error:", response.text)


Returned 20 cancer studies. Top keys: ['protocolSection', 'derivedSection', 'hasResults']


Step 4: Extract Cancer Study Table

In [28]:
records = []
for study in studies:
    protocol = study.get('protocolSection', {})
    id_module = protocol.get('identificationModule', {})
    status_module = protocol.get('statusModule', {})
    conds_module = protocol.get('conditionsModule', {})
    desc_module = protocol.get('descriptionModule', {})
    eligibility_module = protocol.get('eligibilityModule', {})
    sponsor_module = protocol.get('sponsorCollaboratorsModule', {})
    arms_module = protocol.get('armsInterventionsModule', {})
    outcomes_module = protocol.get('outcomesModule', {})

    nct_id = id_module.get('nctId', '')
    brief_title = id_module.get('briefTitle', '')
    status = status_module.get('overallStatus', '')
    conditions = ', '.join(conds_module.get('conditions', []))
    brief_summary = desc_module.get('briefSummary', '')
    eligibility = eligibility_module.get('eligibilityCriteria', '')
    sponsor = sponsor_module.get('leadSponsor', {}).get('name', '')
    interventions = ', '.join([i.get('name', '') for i in arms_module.get('interventions', [])])
    primary_outcomes = ', '.join([o.get('measure', '') for o in outcomes_module.get('primaryOutcomes', [])])

    records.append({
        'nct_id': nct_id,
        'brief_title': brief_title,
        'status': status,
        'conditions': conditions,
        'brief_summary': brief_summary,
        'eligibility_criteria': eligibility,
        'sponsor': sponsor,
        'interventions': interventions,
        'primary_outcomes': primary_outcomes
    })
studies_df = pd.DataFrame(records)
print(studies_df.head(5))
studies_df.to_csv('eda_phase1_cancer.csv', index=False)
print('Saved Cancer table to eda_phase1_cancer.csv')


        nct_id                                        brief_title  \
0  NCT07042243                           The Florida ASCENT Study   
1  NCT00616135  Study of Autologous Fat Enhanced w/ Regenerati...   
2  NCT02012608  Glutamine for the Prevention of Radiation Toxi...   
3  NCT03171025  Adjuvant Nivolumab Following Chemo-Radiation i...   
4  NCT05920876  A Study of QLS32015 in Patients With Recurrent...   

                  status                                         conditions  \
0     NOT_YET_RECRUITING  Cancer, Food Deprivation, Food Habits, Food Se...   
1              COMPLETED  Breast Neoplasms, Carcinoma, Ductal, Breast, M...   
2             TERMINATED                                      Breast Cancer   
3  ACTIVE_NOT_RECRUITING                                     Bladder Cancer   
4                UNKNOWN            Relapsed or Refractory Multiple Myeloma   

                                       brief_summary  \
0  The goal of this clinical trial is to adapt, i...  

EDA Progress Summary and Next Steps
1. Summary of Your Work
You have successfully connected to ClinicalTrials.gov API and systematically extracted data for multiple diseases: Diabetes, Alzheimer's, Asthma, and Cancer.

For each disease, you built rich dataframes containing: trial IDs, titles, status, conditions, brief summaries, eligibility criteria, sponsor, interventions, and primary outcomes.

Exploratory Data Analysis (EDA) has confirmed:

Core clinical fields are available across diseases (and not just for diabetes)

Field variability (sponsors, conditions, outcomes) is clear and documented

No missing values in your essential fields in these samples

You have saved each enriched DataFrame as a reproducible CSV, forming a modular, high-quality evidence corpus for the next project phases.

2. Why This Matters
This approach matches top clinical data science practices—thorough field investigation and transparent, stepwise data building for multiple disease domains.

You now have the evidence base for a scalable retrieval-augmented (RAG) pipeline or robust conversational agent.

Field Distribution & Quality Checks

In [29]:
print(studies_df["sponsor"].value_counts())
print(studies_df["status"].value_counts())
print(studies_df["interventions"].head(15))


sponsor
University of Florida                                     1
Cytori Therapeutics                                       1
University of Arkansas                                    1
University of Utah                                        1
Qilu Pharmaceutical Co., Ltd.                             1
University of Michigan Rogel Cancer Center                1
Lipidica, a.s.                                            1
Qingdao Sino-Cell Biomedicine Co., Ltd.                   1
Aivita Biomedical, Inc.                                   1
Universitaire Ziekenhuizen KU Leuven                      1
The Netherlands Cancer Institute                          1
Peking Union Medical College Hospital                     1
Aydin Adnan Menderes University                           1
University of Aarhus                                      1
Allogene Therapeutics                                     1
Fondazione IRCCS Istituto Nazionale dei Tumori, Milano    1
Zeria Pharmaceutical            

Phase 1 - above


Extracting and analyzing ClinicalTrials.gov data live for multiple diseases (diabetes, Alzheimer's, asthma, cancer).

Exploring key trial metadata fields: identifiers, titles, recruitment status, conditions, summaries, eligibility, sponsors, interventions, and primary outcomes.

Validating data completeness and field variety crucial for personalized retrieval and chatbot explanation.

Saving your datasets for reproducibility, a must for master-level work.

# Rag

Let's build the most crucial part of your healthcare RAG system step-by-step!

We'll:

Chunk your summaries (make evidence 'documents').

Embed those chunks using a simple SentenceTransformer model.

Index them with FAISS.

Search the index with a user query.

In [30]:
!pip install faiss-cpu sentence-transformers numpy pandas


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 46.5 MB/s eta 0:00:00


Step 2: Import And Prepare Your Data

In [35]:
import pandas as pd
from sentence_transformers import SentenceTransformer
import numpy as np
import faiss

# Load your dataset (change filename if needed)
df = pd.read_csv('eda_phase1_cancer.csv')

# For this demo, we'll use only the 'brief_summary' field (could add eligibility, outcome, etc.)
docs = df['brief_summary'].fillna('').tolist()


Step 3: Embed All Chunks

In [36]:
model = SentenceTransformer('all-MiniLM-L6-v2')   # Fast and good for prototyping
embeddings = model.encode(docs)
print('Embeddings shape:', embeddings.shape)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Embeddings shape: (20, 384)


Step 4: Build FAISS Index

In [37]:
embedding_dim = embeddings.shape[1]
faiss_index = faiss.IndexFlatL2(embedding_dim)  # L2 (Euclidean distance) is standard for embeddings
faiss_index.add(np.array(embeddings))
print(f'FAISS index contains {faiss_index.ntotal} vectors.')


FAISS index contains 20 vectors.


Step 5: Query The Index With A User Prompt

In [38]:
query = "What are new treatments for cancer?"  # Change to try your own!
query_vec = model.encode([query])

# Search for closest results
k = 3  # How many results to show
D, I = faiss_index.search(query_vec, k)
print('Nearest document indices:', I)

for i in I[0]:
    print('\n---- Candidate Evidence ----')
    print(df.iloc[i]['brief_title'])
    print(df.iloc[i]['brief_summary'][:400])  # Display first 400 chars of summary
    print(f"NCT ID: {df.iloc[i]['nct_id']}")


Nearest document indices: [[ 5 15  3]]

---- Candidate Evidence ----
Eflornithine to Prevent Cancer in Patients With Barrett's Esophagus
RATIONALE: Chemoprevention therapy is the use of drugs to try and prevent the development or recurrence of cancer. It is not known whether eflornithine is effective in preventing cancer in patients with Barrett's esophagus.

PURPOSE: Randomized double-blinded phase II trial to study the effectiveness of eflornithine in preventing cancer in patients with Barrett's esophagus.
NCT ID: NCT00003076

---- Candidate Evidence ----
hARnessing CAbozantinib and Durvalumab Immuno-oncology Association: ARCADIA Study"
Cabozantinib plus Durvalumab in patients with advanced and chemotherapy-treated bladder carcinoma, of urothelial and non-urothelial histology: an open-label, single-centre, phase 2, single-arm proof-of-concept trial: ARCADIA study
NCT ID: NCT03824691

---- Candidate Evidence ----
Adjuvant Nivolumab Following Chemo-Radiation in Localized Muscle-Invasiv

Experiment

Experiment 1: Chunk Size

In [39]:
# Adjust chunk size here
chunk_size = 300  # Try: 100, 300, 500
chunks = []
chunk_map = []  # Stores (original_idx, chunk) for tracking

for idx, doc in enumerate(df['brief_summary'].fillna('').tolist()):
    for i in range(0, len(doc), chunk_size):
        chunk = doc[i:i+chunk_size]
        if chunk.strip():
            chunks.append(chunk)
            chunk_map.append((idx, chunk))

print(f"Total chunks: {len(chunks)} (original docs: {len(df)})")


Total chunks: 40 (original docs: 20)


In [40]:
# Adjust chunk size here
chunk_size = 100  # Try: 100, 300, 500
chunks = []
chunk_map = []  # Stores (original_idx, chunk) for tracking

for idx, doc in enumerate(df['brief_summary'].fillna('').tolist()):
    for i in range(0, len(doc), chunk_size):
        chunk = doc[i:i+chunk_size]
        if chunk.strip():
            chunks.append(chunk)
            chunk_map.append((idx, chunk))

print(f"Total chunks: {len(chunks)} (original docs: {len(df)})")


Total chunks: 103 (original docs: 20)


In [41]:
# Adjust chunk size here
chunk_size = 500  # Try: 100, 300, 500
chunks = []
chunk_map = []  # Stores (original_idx, chunk) for tracking

for idx, doc in enumerate(df['brief_summary'].fillna('').tolist()):
    for i in range(0, len(doc), chunk_size):
        chunk = doc[i:i+chunk_size]
        if chunk.strip():
            chunks.append(chunk)
            chunk_map.append((idx, chunk))

print(f"Total chunks: {len(chunks)} (original docs: {len(df)})")


Total chunks: 28 (original docs: 20)


In [43]:
chunk_size = 300  # or 500
chunk_overlap = 50  # 10–15% overlap
chunks = []
chunk_map = []
for idx, doc in enumerate(df['brief_summary'].fillna('').tolist()):
    for i in range(0, len(doc), chunk_size - chunk_overlap):
        chunk = doc[i:i+chunk_size]
        if chunk.strip():
            chunks.append(chunk)
            chunk_map.append((idx, chunk))
print(f"Total chunks: {len(chunks)}")


Total chunks: 45


In [44]:
query = "What are new treatments for cancer?"  # You can change to try any medical question
query_embedding = model.encode([query])

# Search the index for top-k similar chunks
k = 3  # Number of answers you want
D, I = faiss_index.search(query_embedding, k)
print(f"Nearest chunk indices: {I}")

for idx in I[0]:
    orig_doc, chunk_txt = chunk_map[idx]  # Which trial the chunk comes from
    print("\n--- Retrieved Evidence ---")
    print("Trial Title:", df.iloc[orig_doc]['brief_title'])
    print("Summary Chunk:", chunk_txt)
    print("NCT ID:", df.iloc[orig_doc]['nct_id'])


Nearest chunk indices: [[ 5 15  3]]

--- Retrieved Evidence ---
Trial Title: The Florida ASCENT Study
Summary Chunk: nt Participants will:

* Participate in ASCENT patient navigator screenings and consultations
* Complete the ASCENT Questionnaire, which comprises the U.S. FSSM, PROMIS-29, and ASA24®
NCT ID: NCT07042243

--- Retrieved Evidence ---
Trial Title: Clinical Performance of Medical Device Software "Lipidica 1.0" for Processing Data Generated by Lipidomic Analysis in Pancreatic Cancer Screening
Summary Chunk: ut at higher risk of this cancer disease due to their predispositions.

Participants will:

* come to baseline and end of study visit for blood sampling and medical imaging
* some participant will undertake one more visit depending on their results on baseline
NCT ID: NCT06549725

--- Retrieved Evidence ---
Trial Title: The Florida ASCENT Study
Summary Chunk: ity.

Phase 1

Patient Participants will:

* Complete the ASCENT Questionnaire, which is comprised of the following

 Standardized RAG Query Code (Change query each time!)

In [45]:
user_queries = [
    "What are new treatments for cancer?",
    "Latest immunotherapy for lung cancer",
    "Eligibility for asthma clinical trials",
    "Guidelines for managing diabetes in elderly patients",
    "Primary outcomes in Alzheimer's studies",
    "Recent advances in pancreatic cancer screening"
]

for query in user_queries:
    print(f"\n=== Query: '{query}' ===")
    query_embedding = model.encode([query])
    D, I = faiss_index.search(query_embedding, 3)  # top 3 evidence chunks
    for idx in I[0]:
        orig_doc, chunk_txt = chunk_map[idx]
        print(f"Trial Title: {df.iloc[orig_doc]['brief_title']}")
        print(f"Summary Chunk: {chunk_txt}")
        print(f"NCT ID: {df.iloc[orig_doc]['nct_id']}")
        print("---")



=== Query: 'What are new treatments for cancer?' ===
Trial Title: The Florida ASCENT Study
Summary Chunk: nt Participants will:

* Participate in ASCENT patient navigator screenings and consultations
* Complete the ASCENT Questionnaire, which comprises the U.S. FSSM, PROMIS-29, and ASA24®
NCT ID: NCT07042243
---
Trial Title: Clinical Performance of Medical Device Software "Lipidica 1.0" for Processing Data Generated by Lipidomic Analysis in Pancreatic Cancer Screening
Summary Chunk: ut at higher risk of this cancer disease due to their predispositions.

Participants will:

* come to baseline and end of study visit for blood sampling and medical imaging
* some participant will undertake one more visit depending on their results on baseline
NCT ID: NCT06549725
---
Trial Title: The Florida ASCENT Study
Summary Chunk: ity.

Phase 1

Patient Participants will:

* Complete the ASCENT Questionnaire, which is comprised of the following:

  * U.S. Food Security Survey Module (U.S. FSSM)
  * Pa

Experiment 2: Change Embedding Model

In [46]:
from sentence_transformers import SentenceTransformer
alt_model = SentenceTransformer('paraphrase-MiniLM-L3-v2')  # Or a domain-specific model if available
alt_chunk_embeddings = alt_model.encode(chunks, show_progress_bar=True)
faiss_index_alt = faiss.IndexFlatL2(alt_chunk_embeddings.shape[1])
faiss_index_alt.add(np.array(alt_chunk_embeddings))

for query in user_queries:
    print(f"\n=== [ALT MODEL] Query: '{query}' ===")
    query_embedding = alt_model.encode([query])
    D, I = faiss_index_alt.search(query_embedding, 3)
    for idx in I[0]:
        orig_doc, chunk_txt = chunk_map[idx]
        print(f"Trial Title: {df.iloc[orig_doc]['brief_title']}")
        print(f"Summary Chunk: {chunk_txt}")
        print(f"NCT ID: {df.iloc[orig_doc]['nct_id']}")
        print("---")


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/2 [00:00<?, ?it/s]


=== [ALT MODEL] Query: 'What are new treatments for cancer?' ===
Trial Title: Eflornithine to Prevent Cancer in Patients With Barrett's Esophagus
Summary Chunk: RATIONALE: Chemoprevention therapy is the use of drugs to try and prevent the development or recurrence of cancer. It is not known whether eflornithine is effective in preventing cancer in patients with Barrett's esophagus.

PURPOSE: Randomized double-blinded phase II trial to study the effectivenes
NCT ID: NCT00003076
---
Trial Title: Whole Body HER3 Quantification With Radiolabelled Patritumab Deruxtecan (HER3-DXd) PET/CT
Summary Chunk: Activity of patritumab deruxtecan (U3-1402; HER3-DXd) has been shown in a phase I/II study in patients with HER3 expressing breast cancer as well as in a phase I study in patients with EGFR TKI refractory EGFR mutation positive NSCLC with a preliminary ORR of 25%. HER3 expression can be seen in mult
NCT ID: NCT06222489
---
Trial Title: Glutamine for the Prevention of Radiation Toxicity in Sub

Downloading a Complete Disease Corpus from ClinicalTrials.gov


Step 1: API Batch Download for Full Results

In [63]:
import requests
import pandas as pd
import time

# Choose your condition (change as needed!)
# condition = "Diabetes"  # replace with any disease you want
# condition = "Cancer"  # replace with any disease you want
# condition = "Asthma"  # replace with any disease you want
# condition = "cardiovascular"  # replace with any disease you want
condition = "Alzheimer"  # replace with any disease you want


base_url = "https://clinicaltrials.gov/api/v2/studies"
params = {
    "query.cond": condition,
    "pageSize": 100  # max per page
}

studies_total = []
page_token = None
request_count = 0

while True:
    if page_token:
        params["pageToken"] = page_token
    print(f"Fetching page {request_count+1}...")
    response = requests.get(base_url, params=params)
    if response.status_code != 200:
        print("Error:", response.text)
        break
    data = response.json()
    studies = data.get('studies', [])
    studies_total.extend(studies)
    page_token = data.get('nextPageToken', None)
    request_count += 1
    if not page_token or len(studies) == 0:
        break
    time.sleep(0.4)  # polite pause for API

print(f"Fetched {len(studies_total)} studies for {condition}.")


Fetching page 1...
Fetching page 2...
Fetching page 3...
Fetching page 4...
Fetching page 5...
Fetching page 6...
Fetching page 7...
Fetching page 8...
Fetching page 9...
Fetching page 10...
Fetching page 11...
Fetching page 12...
Fetching page 13...
Fetching page 14...
Fetching page 15...
Fetching page 16...
Fetching page 17...
Fetching page 18...
Fetching page 19...
Fetching page 20...
Fetching page 21...
Fetching page 22...
Fetching page 23...
Fetching page 24...
Fetching page 25...
Fetching page 26...
Fetching page 27...
Fetching page 28...
Fetching page 29...
Fetching page 30...
Fetching page 31...
Fetching page 32...
Fetching page 33...
Fetching page 34...
Fetching page 35...
Fetching page 36...
Fetching page 37...
Fetching page 38...
Fetched 3761 studies for Alzheimer.


Step 2: Preprocess and Save to DataFrame
Extract the key fields for RAG (as in earlier steps):

In [64]:
records = []
for study in studies_total:
    protocol = study.get('protocolSection', {})
    id_module = protocol.get('identificationModule', {})
    status_module = protocol.get('statusModule', {})
    conds_module = protocol.get('conditionsModule', {})
    desc_module = protocol.get('descriptionModule', {})
    eligibility_module = protocol.get('eligibilityModule', {})
    sponsor_module = protocol.get('sponsorCollaboratorsModule', {})
    arms_module = protocol.get('armsInterventionsModule', {})
    outcomes_module = protocol.get('outcomesModule', {})
    nct_id = id_module.get('nctId', '')
    brief_title = id_module.get('briefTitle', '')
    status = status_module.get('overallStatus', '')
    conditions = ', '.join(conds_module.get('conditions', []))
    brief_summary = desc_module.get('briefSummary', '')
    eligibility = eligibility_module.get('eligibilityCriteria', '')
    sponsor = sponsor_module.get('leadSponsor', {}).get('name', '')
    interventions = ', '.join([i.get('name', '') for i in arms_module.get('interventions', [])])
    primary_outcomes = ', '.join([o.get('measure', '') for o in outcomes_module.get('primaryOutcomes', [])])
    records.append({
        'nct_id': nct_id,
        'brief_title': brief_title,
        'status': status,
        'conditions': conditions,
        'brief_summary': brief_summary,
        'eligibility_criteria': eligibility,
        'sponsor': sponsor,
        'interventions': interventions,
        'primary_outcomes': primary_outcomes
    })
df_full = pd.DataFrame(records)
print(df_full.head())
df_full.to_csv(f"clinical_trials_{condition.lower()}_full.csv", index=False)
print(f"Saved all {condition} trials to clinical_trials_{condition.lower()}_full.csv")


        nct_id                                        brief_title      status  \
0  NCT04451408  A Study of LY3372993 in Participants With Alzh...   COMPLETED   
1  NCT03472183  Exploration of the Enteric Nervous System in A...   WITHDRAWN   
2  NCT00582127  Evaluation of the COGNISION(TM) System as an E...   COMPLETED   
3  NCT07154394  Cardiac Amyloid Deposits and Heart Dysfunction...  RECRUITING   
4  NCT00104273  Rasagiline 1 mg and 2 mg Added to Aricept 10 m...   COMPLETED   

                                          conditions  \
0                         Alzheimer Disease, Healthy   
1                                  Alzheimer Disease   
2                                Alzheimer's Disease   
3  Dementia, Alzheimer Type, Mild Cognitive Impai...   
4                      Dementia, Alzheimer's Disease   

                                       brief_summary  \
0  The main purpose of this study is to evaluate ...   
1  The close homology between the central and ent...   
2  This 

In [65]:
1

1

In [67]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [62]:
1

1

In [68]:
# Example: for diabetes
condition = "Diabetes"
df_full.to_csv(f"/content/drive/MyDrive/Sem 1/LLM/Project/data/clinical_trials_{condition.lower()}_full.csv", index=False)
print(f"Saved {condition} trials to Drive.")


Saved Diabetes trials to Drive.


In [72]:
# condition = "Diabetes"  # replace with any disease you want
# condition = "Cancer"  # replace with any disease you want
# condition = "Asthma"  # replace with any disease you want
# condition = "cardiovascular"  # replace with any disease you want
# condition = "Alzheimer"  # replace with any disease you want

# Example: for diabetes
condition = "Alzheimer"
df_full.to_csv(f"/content/drive/MyDrive/Sem 1/LLM/Project/data/clinical_trials_{condition.lower()}_full.csv", index=False)
print(f"Saved {condition} trials to Drive.")


Saved Alzheimer trials to Drive.


In [75]:
import pandas as pd
for condition in ["diabetes", "cancer", "asthma", "cardiovascular", "alzheimer"]:
    file_path = f"/content/drive/MyDrive/Sem 1/LLM/Project/data/clinical_trials_{condition}_full.csv"
    df = pd.read_csv(file_path)
    print(f"{condition.title()} trials:", len(df))
    print("Unique conditions:", df['conditions'].unique()[:5])
    print("Example title:", df.loc[0, 'brief_title'])
    print('---')


Diabetes trials: 22868
Unique conditions: ['Type 2 Diabetes, Overweight' 'Diabetes, Diabetes Mellitus, Type 2'
 'Diabetes Mellitus, Type 2' 'Diabetes Mellitus'
 'PreDiabetes, Obesity, Morbid, Bariatric Surgery Candidate']
Example title: Diabetes and Metabolic Postprandial Responses
---
Cancer trials: 114834
Unique conditions: ['Cancer, Food Deprivation, Food Habits, Food Selection, Colorectal Cancer, Prostate Cancer, Lung Cancer, Breast Cancer, Gynecologic Cancer, Hematologic Cancer, Skin Cancer, Melanoma, Nutrition Poor, Nutritional Deficiency'
 'Breast Neoplasms, Carcinoma, Ductal, Breast, Mammaplasty, Mastectomy, Segmental, Lumpectomy, Breast Reconstruction,'
 'Breast Cancer' 'Bladder Cancer'
 'Relapsed or Refractory Multiple Myeloma']
Example title: The Florida ASCENT Study
---
Asthma trials: 5038
Unique conditions: ['Respiratory Diseases, Lung Diseases, Cystic Fibrosis, COPD, Asthma'
 'Moderate to Severe Asthma' 'Status Asthmaticus'
 'Respiratory Distress Syndrome, Pneumonia, Bron

In [76]:
1

1

Combine

In [77]:
import pandas as pd
# Load individual CSVs
files = [
    '/content/drive/MyDrive/Sem 1/LLM/Project/data/clinical_trials_diabetes_full.csv',
    '/content/drive/MyDrive/Sem 1/LLM/Project/data/clinical_trials_cancer_full.csv',
    '/content/drive/MyDrive/Sem 1/LLM/Project/data/clinical_trials_asthma_full.csv',
    '/content/drive/MyDrive/Sem 1/LLM/Project/data/clinical_trials_cardiovascular_full.csv',
    '/content/drive/MyDrive/Sem 1/LLM/Project/data/clinical_trials_alzheimer_full.csv'
]
all_dfs = [pd.read_csv(fp) for fp in files]
master_df = pd.concat(all_dfs, ignore_index=True)
print(f"Total records in master: {len(master_df)}")

master_df.to_csv('/content/drive/MyDrive/Sem 1/LLM/Project/data/clinical_trials_master_full.csv', index=False)
print("Saved combined CSV to Drive!")


Total records in master: 209693
Saved combined CSV to Drive!


In [78]:
1

1

Chunking

In [80]:
import pandas as pd
master_df = pd.read_csv('/content/drive/MyDrive/Sem 1/LLM/Project/data/clinical_trials_master_full.csv')
chunk_size, chunk_overlap = 300, 50
chunks, chunk_map = [], []
for idx, row in master_df.iterrows():
    for field in ['brief_summary', 'eligibility_criteria', 'primary_outcomes']:
        text = str(row.get(field, '')).strip()
        if text:
            for i in range(0, len(text), chunk_size - chunk_overlap):
                sub_chunk = text[i:i+chunk_size]
                if sub_chunk:
                    chunks.append(sub_chunk)
                    chunk_map.append({
                        'doc_idx': idx,
                        'field': field,
                        'chunk': sub_chunk,
                        'nct_id': row['nct_id'],
                        'title': row['brief_title'],
                        'conditions': row['conditions'],
                        'status': row['status']
                    })
print(f"Total chunks: {len(chunks)}")


Total chunks: 2505925


In [81]:
1

1

Step 1: Embedding and Retrieval Code

In [82]:
from sentence_transformers import SentenceTransformer
import numpy as np
import faiss

# 1. Embed chunks
model = SentenceTransformer('all-MiniLM-L6-v2')  # swap for bio model later if needed
embeddings = model.encode(chunks, show_progress_bar=True)

# 2. Build FAISS index
embedding_dim = embeddings.shape[1]
faiss_index = faiss.IndexFlatL2(embedding_dim)
faiss_index.add(np.array(embeddings))
print(f"Index contains {faiss_index.ntotal} chunks")

# 3. Sample retrieval query (change as needed)
query = "What are new treatments for diabetes?"  # adapt this for demo/other diseases
query_embedding = model.encode([query])

k = 5  # how many results to retrieve
D, I = faiss_index.search(query_embedding, k)
print(f"Nearest chunk indices: {I}")

for idx in I[0]:
    info = chunk_map[idx]
    print("\n--- Retrieved Evidence ---")
    print(f"Disease: {info['conditions']}")
    print(f"Field: {info['field']} | Title: {info['title']}")
    print(f"Chunk: {info['chunk'][:400]}")
    print(f"NCT ID: {info['nct_id']}")


Batches:   0%|          | 0/78311 [00:00<?, ?it/s]

KeyboardInterrupt: 

Filter your chunks and run retrieval on a subset

In [83]:
# Chunking - for fixed character size and overlap
chunk_size = 400          # You can use 300, 400, or 500
chunk_overlap = 60        # 10–15% recommended overlap
chunks = []
chunk_map = []  # Always update this with metadata for auditing!

for idx, row in master_df.iterrows():
    for field in ['brief_summary', 'eligibility_criteria', 'primary_outcomes']:
        text = str(row.get(field, '')).strip()
        if text:
            for i in range(0, len(text), chunk_size - chunk_overlap):
                sub_chunk = text[i:i+chunk_size]
                if sub_chunk:
                    chunks.append(sub_chunk)
                    chunk_map.append({
                        'doc_idx': idx,
                        'field': field,
                        'chunk': sub_chunk,
                        'nct_id': row['nct_id'],
                        'title': row['brief_title'],
                        'conditions': row['conditions'],
                        'status': row['status']
                    })
print(f"Total chunks: {len(chunks)}")


Total chunks: 1947088


In [85]:
# Example code for filtering chunks (adjust as needed):
filtered_indices = [i for i, c in enumerate(chunk_map)
                    if 'Diabetes' in c['conditions'] and c['field']=='brief_summary']
chunks_subset = [chunks[i] for i in filtered_indices]
chunk_map_subset = [chunk_map[i] for i in filtered_indices]

print(f"Filtered chunks count: {len(chunks_subset)}")

# Then embed and index this subset
from sentence_transformers import SentenceTransformer
import numpy as np
import faiss

model = SentenceTransformer('all-MiniLM-L6-v2')  # or swap for biomedical model
embeddings = model.encode(chunks_subset, show_progress_bar=True)

embedding_dim = embeddings.shape[1]
faiss_index = faiss.IndexFlatL2(embedding_dim)
faiss_index.add(np.array(embeddings))
print(f"FAISS index contains {faiss_index.ntotal} chunks")

# Sample query
query = "What are new treatments for diabetes?"
query_embedding = model.encode([query])
k = 5
D, I = faiss_index.search(query_embedding, k)

for idx in I[0]:
    info = chunk_map_subset[idx]
    print("\n--- Retrieved Evidence ---")
    print(f"Disease: {info['conditions']}")
    print(f"Field: {info['field']} | Title: {info['title']}")
    print(f"Chunk: {info['chunk'][:400]}")
    print(f"NCT ID: {info['nct_id']}")


TypeError: argument of type 'float' is not iterable

In [86]:
filtered_indices = [
    i for i, c in enumerate(chunk_map)
    if isinstance(c['conditions'], str) and 'Diabetes' in c['conditions'] and c['field'] == 'brief_summary'
]
chunks_subset = [chunks[i] for i in filtered_indices]
chunk_map_subset = [chunk_map[i] for i in filtered_indices]
print(f"Filtered chunk count: {len(chunks_subset)}")


Filtered chunk count: 52965


In [87]:
filtered_indices = []
for i, c in enumerate(chunk_map):
    cond = c['conditions']
    if isinstance(cond, str):
        if 'Diabetes' in cond and c['field']=='brief_summary':
            filtered_indices.append(i)

chunks_subset = [chunks[i] for i in filtered_indices]
chunk_map_subset = [chunk_map[i] for i in filtered_indices]
print(f"Filtered chunk count: {len(chunks_subset)}")


Filtered chunk count: 52965


In [88]:
from sentence_transformers import SentenceTransformer
import numpy as np
import faiss

model = SentenceTransformer('all-MiniLM-L6-v2')  # Or a biomedical model if you want to try later
embeddings = model.encode(chunks_subset, show_progress_bar=True)

faiss_index = faiss.IndexFlatL2(embeddings.shape[1])
faiss_index.add(np.array(embeddings))
print(f"FAISS index contains {faiss_index.ntotal} diabetes summary chunks")


Batches:   0%|          | 0/1656 [00:00<?, ?it/s]

FAISS index contains 52965 diabetes summary chunks


In [89]:
query = "What are new treatments for diabetes?"
query_embedding = model.encode([query])
k = 5  # Number of top results to show
D, I = faiss_index.search(query_embedding, k)
print(f"Nearest chunk indices: {I}")

for idx in I[0]:
    info = chunk_map_subset[idx]
    print(f"\n--- Retrieved Evidence ---")
    print(f"Disease: {info['conditions']}")
    print(f"Field: {info['field']} | Title: {info['title']}")
    print(f"Chunk: {info['chunk'][:400]}")
    print(f"NCT ID: {info['nct_id']}")


Nearest chunk indices: [[28813  9911  3737  5592 35639]]

--- Retrieved Evidence ---
Disease: Diabetes Mellitus, Diabetes Mellitus, Type 2
Field: brief_summary | Title: The PACT (Patient Activation Through Conversations) Study
Chunk:  diabetes treatment.
NCT ID: NCT06444074

--- Retrieved Evidence ---
Disease: Type 2 Diabetes
Field: brief_summary | Title: Laparoscopic Bariatric Surgery to Treat Type 2 Diabetes in Obese Patients
Chunk: reatment with a combination of drugs, diet, and lifestyle changes for control of type 2 diabetes.
NCT ID: NCT00428571

--- Retrieved Evidence ---
Disease: Type 1 Diabetes (T1D)
Field: brief_summary | Title: Cohort Study to Refine the Positioning of Closed-loop Therapy Versus Islet Transplantation in the Management of Patients With Unstable Type 1 Diabetes
Chunk:  for management of unstable diabetes.
NCT ID: NCT07006272

--- Retrieved Evidence ---
Disease: Type 2 Diabetes Mellitus, Diabetes
Field: brief_summary | Title: Dose Escalation Study to Evaluate Sa

In [96]:
import pandas as pd
results = []
for idx in I[0]:
    info = chunk_map_subset[idx]
    results.append({
        'query': query,
        'disease': info['conditions'],
        'field': info['field'],
        'trial_title': info['title'],
        'evidence_chunk': info['chunk'],
        'nct_id': info['nct_id']
    })

results_df = pd.DataFrame(results)
results_df.to_csv('/content/drive/MyDrive/Sem 1/LLM/Project/data/diabetes_rag_query_results.csv', index=False)
print("Saved results to your Drive!")


Saved results to your Drive!


In [97]:
1

1

In [99]:
import numpy as np
import pandas as pd

# After embedding
np.save('/content/drive/MyDrive/Sem 1/LLM/Project/data/diabetes_chunk_embeddings.npy', embeddings)
pd.DataFrame(chunk_map_subset).to_pickle('/content/drive/MyDrive/Sem 1/LLM/Project/data/diabetes_chunk_map.pkl')
print("Saved embeddings and metadata to Drive!")


Saved embeddings and metadata to Drive!


In [100]:
1

1

In [ ]:
# for later on

# import numpy as np
# import pandas as pd

# embeddings = np.load('/content/drive/MyDrive/diabetes_chunk_embeddings.npy')
# chunk_map_subset = pd.read_pickle('/content/drive/MyDrive/diabetes_chunk_map.pkl').to_dict('records')


# import faiss
# faiss_index = faiss.IndexFlatL2(embeddings.shape[1])
# faiss_index.add(embeddings)


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch

# Example: Use Mistral-family model (swap for Llama2, Gemma, Phi-2, etc)
model_name = "mistralai/Mistral-7B-Instruct-v0.2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, device_map="auto")

# Compose RAG prompt
retrieved_chunks = ["Chunk 1 text", "Chunk 2 text", "Chunk 3 ..."]  # Use your real chunk texts
context = "\n\n".join(retrieved_chunks)
query = "What are new treatments for diabetes?"
prompt = f"You are a clinical research assistant. Use ONLY the following CONTEXT to answer the QUESTION and cite trial NCT IDs.\n\nCONTEXT:\n{context}\n\nQUESTION:\n{query}\n\nAnswer:"

# Generate answer
generator = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=200)
output = generator(prompt)
print(output[0]['generated_text'])


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

Device set to use cpu
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [ ]:
1

In [ ]:
1

In [ ]:
1

In [ ]:
1